In [ ]:
write_images = False


wirte_output_txt = False 
# Specify everytime Simulation is called 
# WARNING --> Set to False when running more then 10 simulations 
#            (otherwise it will be super slow and might crash)

In [ ]:
import numpy as np

In [ ]:
from emukit.core import ContinuousParameter, ParameterSpace
from emukit.core.initial_designs import RandomDesign

import GPy
from GPy.models import GPRegression
from emukit.model_wrappers import GPyModelWrapper
from emukit.sensitivity.monte_carlo import MonteCarloSensitivity

import matplotlib.pyplot as plt
import mlai.plot as plot

In [ ]:
%run Missile_utils.ipynb

In [ ]:
simulation_output = 'range' 
# We divide by 1000 to avoid dealing with too large numbers

We consider missiles with only 1 stage

In [ ]:

basic_param_spaces = {
    'payload':  [10, 2410],
    'missilediam':  [0.1, 9.9],
    'rvdiam':  [0.1, 9.9],
    'estrange': [100, 4900], 
    'fuelmass': [500, 6000], # [500, 7000], 
    'drymass':  [1000, 3000],
    'Isp0':  [100, 800],# [100, 800],
    'thrust0':  [10000, 69000],
}

In [ ]:
from sklearn.metrics import mean_squared_error
import math

def compute_rmse(y_actual, y_predicted):
    MSE = mean_squared_error(y_actual, y_predicted)
    RMSE = math.sqrt(MSE)
 
    return RMSE

def evaluate_prediction(y_actual, y_predicted):
    return compute_rmse(y_actual, y_predicted)
    
    

# 0. Only one param - m0

In [ ]:
m0_param_1 = 'fuelmass'
m0_domain_param_1 = basic_param_spaces[m0_param_1] 

m0_space = ParameterSpace(
          [ContinuousParameter(m0_param_1, *m0_domain_param_1), 
          ])

custom_param_names = [m0_param_1]

In [ ]:
def run_missile_sim(custom_params):
    """
    Recives in input an array of custom parameters.
    Each row represents a set of different parameters
    Each column is a different parameter (#cols = len(custom_param_names))
    """
    default_params_IRAQ = {
        'payload':500,
        'missilediam':0.88,
        'rvdiam':0,
        'estrange':600,
        'numstages':1,
        'fuelmass':[0,5600],
        'drymass':[0,1200],
        'Isp0':[0,226],
        'thrust0':[0,9177.4]
    }
    
    
    y = np.zeros((custom_params.shape[0], 1))
    for i in range(custom_params.shape[0]):
        params_to_use = default_params_IRAQ
        # Overwrite default param variables
        for j in range(custom_params.shape[1]):
            param_name = custom_param_names[j]
            if param_name in ['fuelmass', 'drymass', 'Isp0', 'thrust0']:
                params_to_use[param_name][1] = custom_params[i,j] 
            else:
                params_to_use[param_name] = custom_params[i, j]
        
            if j==0:
                print('\nNew simulation \n')
            str_to_print = param_name + ': ' + str(custom_params[i,j])
            print(str_to_print)
             
                
        # Run simulation
        output_path = 'results/results_' + str(i) + '.txt'
        sim_output = run_one_sim(
            numstages=params_to_use["numstages"], 
            fuelmass=params_to_use["fuelmass"], 
            drymass=params_to_use["drymass"], 
            thrust0=params_to_use["thrust0"], 
            Isp0=params_to_use["Isp0"], 
            payload=params_to_use["payload"],  
            missilediam=params_to_use["missilediam"],  
            rvdiam=params_to_use["rvdiam"], 
            est_range=params_to_use["estrange"], 
            output_path=output_path, 
            simulation_output=simulation_output,
        )
        
        y[i, 0] = sim_output
    return y


In [ ]:
# Get true points (to build model)
wirte_output_txt = True

m0_design = RandomDesign(m0_space)
m0_x = m0_design.get_samples(3)
m0_y = run_missile_sim(m0_x)

In [ ]:
# Build model
m0_var_kernel = (100)**2 
m0_lengthscale = 100 # 1
m0_var_linear_kernel = (100)**2 
m0_var_noise = 1e-5 # small value

#kern = GPy.kern.RBF(input_dim=1, lengthscale=100, variance =var_kernel )  # , lengthscale=0.08, variance=20
# kern = GPy.kern.Matern32(input_dim=1)
# kern = GPy.kern.Linear(input_dim=1)


constrain_lengthscale = False

m0_rbf_kern = GPy.kern.RBF(input_dim=1, lengthscale=m0_lengthscale)
if constrain_lengthscale:
    m0_rbf_kern.lengthscale.constrain_bounded(m0_lengthscale, m0_lengthscale*1e12)

m0_kern = m0_rbf_kern + \
    GPy.kern.Linear(input_dim=1)


m0_model_gpy = GPRegression(m0_x,m0_y, kernel=m0_kern)
m0_model_gpy.kern.variance =  m0_var_kernel 
m0_model_gpy.likelihood.variance.fix(m0_var_noise)  

display(m0_model_gpy)



In [ ]:
# Fit emulator
m0_model_emukit = GPyModelWrapper(m0_model_gpy)
m0_model_emukit.optimize() # Optimize model hyperparameters



In [ ]:
display(m0_model_gpy)

In [ ]:
# Get true points corresponding to param_1_x_plot (for plot)
wirte_output_txt = False

nr_points_plot = 301
m0_param_1_x_plot = np.linspace(m0_space.parameters[0].min, m0_space.parameters[0].max, nr_points_plot)[:, None]
m0_param_1_y_plot = run_missile_sim(m0_param_1_x_plot)



In [ ]:
# Get model prediction on param_1_x_plot
m0_mu_plot, m0_var_plot = m0_model_emukit.predict(m0_param_1_x_plot)



In [ ]:
# Plot
def helper_plot_emulator_errorbars(x_plot, y_plot, mu_plot, var_plot, model_emukit):
    """Helper function for plotting the emulator fit."""
    ax.plot(model_emukit.X[:, 0], model_emukit.Y, 'ro', markersize=10, label='observations')
    ax.plot(x_plot[:, 0], mu_plot, 'C0', label='model', linewidth=3)
    ax.plot(x_plot[:, 0], y_plot, 'k', label='target function', linewidth=2)
#     ax.fill_between(x_plot[:, index],
#                  mu_plot[:, 0] + np.sqrt(var_plot)[:, 0],
#                  mu_plot[:, 0] - np.sqrt(var_plot)[:, 0], color='C0', alpha=0.6)
    ax.fill_between(x_plot[:, 0],
                 mu_plot[:, 0] + 2 * np.sqrt(var_plot)[:, 0],
                 mu_plot[:, 0] - 2 * np.sqrt(var_plot)[:, 0], color='C0', alpha=0.4)
#     ax.fill_between(x_plot[:, index],
#                  mu_plot[:, 0] + 3 * np.sqrt(var_plot)[:, 0],
#                  mu_plot[:, 0] - 3 * np.sqrt(var_plot)[:, 0], color='C0', alpha=0.2)
    ax.legend(loc=2)
    ax.set_xlabel(custom_param_names[0])
    ax.set_ylabel('$f(x)$')
    ax.grid(True)
    #ax.set_xlim(-0.01, 1)
    #ax.set_ylim([-20, 20])
    

In [ ]:
fig, ax = plt.subplots(figsize=plot.big_wide_figsize)
helper_plot_emulator_errorbars(x_plot=m0_param_1_x_plot, y_plot=m0_param_1_y_plot, 
                               mu_plot=m0_mu_plot, var_plot=m0_var_plot, 
                               model_emukit=m0_model_emukit)

m0_rmse = evaluate_prediction(y_actual=m0_param_1_y_plot, y_predicted=m0_mu_plot)
print("RMSE m0 (pre experiment design loop): ", m0_rmse)

### Experiment design loop

In [ ]:
from emukit.experimental_design.experimental_design_loop import ExperimentalDesignLoop
from emukit.experimental_design.acquisitions import IntegratedVarianceReduction, ModelVariance

In [ ]:
m0_2_model_emukit = m0_model_emukit

In [ ]:
wirte_output_txt = False

integrated_variance = IntegratedVarianceReduction(space=m0_space,
                                                  model=m0_2_model_emukit)
m0_ed = ExperimentalDesignLoop(space=m0_space, 
                            model=m0_2_model_emukit, 
                            acquisition = integrated_variance,
                            batch_size = 1) 
# bach size is set to one in this example as we’ll collect evaluations 
# sequentially but parallel evaluations are allowed
m0_ed.run_loop(user_function=run_missile_sim, stopping_condition=5)



In [ ]:
m0_2_mu_plot, m0_2_var_plot = m0_2_model_emukit.predict(m0_param_1_x_plot)



In [ ]:
fig, ax = plt.subplots(figsize=plot.big_wide_figsize)
helper_plot_emulator_errorbars(x_plot=m0_param_1_x_plot, y_plot=m0_param_1_y_plot, 
                               mu_plot=m0_2_mu_plot, var_plot=m0_2_var_plot, 
                               model_emukit=m0_2_model_emukit)

m0_2_rmse = evaluate_prediction(y_actual=m0_param_1_y_plot, y_predicted=m0_2_mu_plot)
print("RMSE m0 (post experiment design loop): ", m0_2_rmse)

# 0. Only one param - m1

In [ ]:
m1_param_1 = 'Isp0'
m1_domain_param_1 = basic_param_spaces[m1_param_1] # [500, 6000] # [5000,15000]

m1_space = ParameterSpace(
          [ContinuousParameter(m1_param_1, *m1_domain_param_1), 
          ])

custom_param_names = [m1_param_1]

In [ ]:
def run_missile_sim(custom_params):
    """
    Recives in input an array of custom parameters.
    Each row represents a set of different parameters
    Each column is a different parameter (#cols = len(custom_param_names))
    """
    default_params_IRAQ = {
        'payload':500,
        'missilediam':0.88,
        'rvdiam':0,
        'estrange':600,
        'numstages':1,
        'fuelmass':[0,5600],
        'drymass':[0,1200],
        'Isp0':[0,226],
        'thrust0':[0,9177.4]
    }
    
    
    y = np.zeros((custom_params.shape[0], 1))
    for i in range(custom_params.shape[0]):
        params_to_use = default_params_IRAQ
        # Overwrite default param variables
        for j in range(custom_params.shape[1]):
            param_name = custom_param_names[j]
            if param_name in ['fuelmass', 'drymass', 'Isp0', 'thrust0']:
                params_to_use[param_name][1] = custom_params[i,j] # OK as long as we are considering missiles with only 1 stage
            else:
                params_to_use[param_name] = custom_params[i, j]
        
            ## TEMP ## Better customise this
            if j==0:
                print('\nNew simulation \n')
            str_to_print = param_name + ': ' + str(custom_params[i,j])
            print(str_to_print)
            ## 
                
        # Run simulation
        output_path = 'results/results_' + str(i) + '.txt' # TODO Define better identifier
        sim_output = run_one_sim(
            numstages=params_to_use["numstages"], 
            fuelmass=params_to_use["fuelmass"], 
            drymass=params_to_use["drymass"], 
            thrust0=params_to_use["thrust0"], 
            Isp0=params_to_use["Isp0"], 
            payload=params_to_use["payload"],  
            missilediam=params_to_use["missilediam"],  
            rvdiam=params_to_use["rvdiam"], 
            est_range=params_to_use["estrange"], 
            output_path=output_path, 
            simulation_output=simulation_output,
        )
        
        y[i, 0] = sim_output
    return y


In [ ]:
# Get true points (to build model)
wirte_output_txt = True

m1_design = RandomDesign(m1_space)
m1_x = m1_design.get_samples(3)
m1_y = run_missile_sim(m1_x)

In [ ]:
# Build model
m1_var_kernel = (100)**2 
m1_lengthscale = 100 # 1
m1_var_linear_kernel = (100)**2 
m1_var_noise = 1e-5 # small value

constrain_lengthscale = True

#kern = GPy.kern.RBF(input_dim=1, lengthscale=100, variance =var_kernel )  # , lengthscale=0.08, variance=20
# kern = GPy.kern.Matern32(input_dim=1)
# kern = GPy.kern.Linear(input_dim=1)
m1_rbf_kern = GPy.kern.RBF(input_dim=1, lengthscale=m1_lengthscale)
if constrain_lengthscale:
    m1_rbf_kern.lengthscale.constrain_bounded(m1_lengthscale, m1_lengthscale*1e12)

m1_kern = m1_rbf_kern + \
    GPy.kern.Linear(input_dim=1)
# m1_kern = m1_rbf_kern 

m1_model_gpy = GPRegression(m1_x,m1_y, kernel=m1_kern)
m1_model_gpy.kern.variance =  m1_var_kernel 
m1_model_gpy.likelihood.variance.fix(m1_var_noise)  
display(m1_model_gpy)



In [ ]:
# Fit emulator
m1_model_emukit = GPyModelWrapper(m1_model_gpy)
m1_model_emukit.optimize() # Optimize model hyperparameters



In [ ]:
display(m1_model_gpy)

In [ ]:
# Get true points corresponding to param_1_x_plot (for plot)
wirte_output_txt = False

nr_points_plot = 301
m1_param_1_x_plot = np.linspace(m1_space.parameters[0].min, m1_space.parameters[0].max, nr_points_plot)[:, None]
m1_param_1_y_plot = run_missile_sim(m1_param_1_x_plot)



In [ ]:
# Get model prediction on param_1_x_plot
m1_mu_plot, m1_var_plot = m1_model_emukit.predict(m1_param_1_x_plot)



In [ ]:
fig, ax = plt.subplots(figsize=plot.big_wide_figsize)
helper_plot_emulator_errorbars(x_plot=m1_param_1_x_plot, y_plot=m1_param_1_y_plot, 
                               mu_plot=m1_mu_plot, var_plot=m1_var_plot, 
                               model_emukit=m1_model_emukit)

m1_rmse = evaluate_prediction(y_actual=m1_param_1_y_plot, y_predicted=m1_mu_plot)
print("RMSE m1 (pre experiment design loop): ", m1_rmse)

### Experiment design loop

In [ ]:
m1_2_model_emukit = m1_model_emukit

In [ ]:
wirte_output_txt = False

integrated_variance = IntegratedVarianceReduction(space=m1_space,
                                                  model=m1_2_model_emukit)
m1_ed = ExperimentalDesignLoop(space=m1_space, 
                            model=m1_2_model_emukit, 
                            acquisition = integrated_variance,
                            batch_size = 1) 
# bach size is set to one in this example as we’ll collect evaluations 
# sequentially but parallel evaluations are allowed
m1_ed.run_loop(user_function=run_missile_sim, stopping_condition=5)



In [ ]:
m1_2_mu_plot, m1_2_var_plot = m1_2_model_emukit.predict(m1_param_1_x_plot)



In [ ]:
fig, ax = plt.subplots(figsize=plot.big_wide_figsize)
helper_plot_emulator_errorbars(x_plot=m1_param_1_x_plot, y_plot=m1_param_1_y_plot, 
                               mu_plot=m1_2_mu_plot, var_plot=m1_2_var_plot, 
                               model_emukit=m1_2_model_emukit)

m1_2_rmse = evaluate_prediction(y_actual=m1_param_1_y_plot, y_predicted=m1_2_mu_plot)
print("RMSE m1 (post experiment design loop): ", m1_2_rmse)